In [178]:
import pandas as pd
import numpy as np

In [179]:
import sys
sys.path.append('../ames') # path the the directory
import config
from data_prep import clean, add_features, dummify

In [180]:
housing = clean(config.HOUSING_CSV)
housing = add_features(housing)
# housing = pd.read_pickle(config.HOUSING_PICKLE)

In [181]:
housing

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,TotalLivingArea,UnusedLotSize,HasPool,Toilets,Showers,DecadeBuilt,DecadeRemodel,HouseAge,HouseAgeSq,HouseAgeLog
0,909176150,856,126000,30,RL,0.0,7890,Pave,NaN,Reg,...,1094.0,7034,False,2.0,2.0,1930,1930,72,5184,4.276666
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,...,1994.0,3186,False,3.0,3.0,1980,1980,27,729,3.295837
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,...,1738.0,5059,False,1.0,1.0,1930,1930,81,6561,4.394449
3,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,...,1039.0,7429,False,1.0,1.0,1900,1900,111,12321,4.709530
4,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,...,2308.0,7590,False,4.0,3.0,2000,2000,10,100,2.302585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,903205040,952,121000,30,RL,0.0,8854,Pave,NaN,Reg,...,952.0,7902,False,1.0,1.0,1910,1910,95,9025,4.553877
2576,905402060,1733,139600,20,RL,0.0,13680,Pave,NaN,IR1,...,1733.0,11947,False,2.0,2.0,1950,1950,56,3136,4.025352
2577,909275030,2002,145000,90,RH,82.0,6270,Pave,NaN,Reg,...,2286.0,5269,False,2.0,2.0,1940,1940,62,3844,4.127134
2578,907192040,1842,217500,60,RL,0.0,8826,Pave,NaN,Reg,...,2683.0,7841,False,4.0,3.0,2000,2000,11,121,2.397895


In [182]:
limited_variables = ['TotalLivingArea','UnusedLotSize','HasPool','OverallQual','GarageArea','YearBuilt']
top_unique_features = ['TotalLivingArea','YearRemodAdd','GarageArea','TotalBsmtSF',
                       'FullBath','UnusedLotSize','MasVnrArea','MoSold','LotFrontage','Fireplaces',
                       'TotRmsAbvGrd', 'OverallQual','HouseAge', 'HouseAgeSq']
X = housing[top_unique_features]
dummy_vars = {'Neighborhood':'Nbhd','LotConfig':'LC','SaleCondition':'SC'}
dummy_df = dummify(housing, dummy_vars)
X = pd.concat([X,dummy_df], axis=1)
y = np.log(housing['SalePrice'])

In [183]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [194]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_features='sqrt', random_state=42)
rfr.fit(X_train,y_train)
print(rfr.score(X_train, y_train))
print(rfr.score(X_test,y_test))

0.9816908106924529
0.897832553961976


<p>0.9840305848772934</p>
<p>0.9023077469434058</p>
<p>From ALL the variables</p>
<p>24412.676585562695</p>

In [185]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(y_test, rfr.predict(X_test), squared=False)
print(rms)

0.12482340748412257


## Feature Importance

In [186]:
top_unique_features = ['OverallQual','TotalLivingArea','YearRemodAdd','GarageArea','TotalBsmtSF',
                       'FullBath','UnusedLotSize','MasVnrArea','Fireplaces',
                       'TotRmsAbvGrd', 'HouseAgeSq', 'MoSold', 'LotFrontage']

In [187]:
d = pd.DataFrame(rfr.feature_importances_,index=X.columns,columns=['Importance'])
d.sort_values('Importance',ascending=False).head(25)

,Importance
OverallQual,0.160856
TotalLivingArea,0.144628
HouseAgeSq,0.100005
HouseAge,0.096655
GarageArea,0.088506
TotalBsmtSF,0.068136
YearRemodAdd,0.050150
FullBath,0.045460
Fireplaces,0.043582
UnusedLotSize,0.039913


In [188]:
rfr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [189]:
# Number of trees in random forest
n_estimators = [40,100] #[int(x) for x in np.linspace(start=10, stop=80, num=10)]
# Maximum number of levels in tree
max_depth = [i*2+4 for i in range(1,5)] + [None]
# Minimum number of samples required to split a node
min_samples_split = [1,2,3,4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2 ]
# Method of selecting samples for training each tree
max_features = [8, 9, 10, 11] 
# Method of selecting number of features    

bootstrap = [True]

param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap,
                'max_features' : max_features}

In [190]:
%timeit
rfc = RandomForestRegressor(random_state=42)
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rfc, param_grid=param_grid, cv=5, verbose=2, n_jobs=4)
rf_Grid.fit(X_train,y_train)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 444 tasks      | elapsed:   11.1s
[Parallel(n_jobs=4)]: Done 1256 tasks      | elapsed:   36.0s
/Users/janruffner/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done 1600 out of 1600 | elapsed:   49.0s finished


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=4,
             param_grid={'bootstrap': [True], 'max_depth': [6, 8, 10, 12, None],
                         'max_features': [8, 9, 10, 11],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [1, 2, 3, 4],
                         'n_estimators': [40, 100]},
             verbose=2)

In [191]:
rf_Grid.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 100}

rfr = RandomForestRegressor(max_depth=None,
                            min_samples_split=2, 
                            min_samples_leaf=1,
                            n_estimators=100,
                            max_features=9, 
                            random_state=42)
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))

In [192]:
rfr = RandomForestRegressor(max_depth=None, 
                            min_samples_split=3, 
                            min_samples_leaf=1, 
                            n_estimators=100, 
                            max_features=10, 
                            random_state=42) 

rfr.fit(X_train, y_train) 
print(rfr.score(X_train, y_train))

0.978217429585285


In [193]:
print(rfr.score(X_test, y_test))

0.9050364273002776
